In [ ]:
import geopandas as gpd
import pandas as pd

points = gpd.read_file(
    "points.gpkg", layer="kappazunder_image_punkte__ogdwienkappazunderimagepogd"
)
points = points.set_index("image_name")

trajectories = gpd.read_file("trajectories.gpkg")

In [ ]:
import sqlite3

conn = sqlite3.connect("download-state.db")
conn.row_factory = sqlite3.Row
cur = conn.cursor()

cur.execute("PRAGMA journal_mode=WAL;")

cur.execute(
    """
CREATE TABLE IF NOT EXISTS trajectories (
    trajectory_id TEXT PRIMARY KEY,

    download_id TEXT,
    download_bytes INTEGER,
    download_expires_at TEXT,

    is_sensor1_completed INTEGER DEFAULT 0,
    is_sensor2_completed INTEGER DEFAULT 0,
    is_sensor3_completed INTEGER DEFAULT 0,
    is_sensor4_completed INTEGER DEFAULT 0,

    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""
)

if cur.execute("SELECT 1 FROM trajectories LIMIT 1;").fetchone() is None:
    for row in trajectories.itertuples():
        cur.execute(
            """
            INSERT OR IGNORE INTO trajectories (trajectory_id)
            VALUES (?);
        """,
            (row.trajectoryid,),
        )
    print("No download-state.db found, initialized with trajectory IDs.")

conn.commit()

In [ ]:
download_state_df = pd.read_sql_query("SELECT * FROM trajectories;", conn)

if "download_expires_at" in download_state_df.columns:
    download_state_df["download_expires_at"] = pd.to_datetime(
        download_state_df["download_expires_at"]
    )

trajectories = trajectories.merge(
    download_state_df, how="left", left_on="trajectoryid", right_on="trajectory_id"
)

trajectories = trajectories.drop(columns=["trajectory_id"])

for col in ["download_id", "download_bytes", "download_expires_at"]:
    if col in trajectories.columns:
        trajectories[col] = trajectories[col].where(trajectories[col].notna(), None)

for col in [f"is_sensor{i}_completed" for i in range(1, 5)]:
    if col in trajectories.columns:
        trajectories[col] = trajectories[col].fillna(0).astype(int)

print("SQLite state merged into trajectories DataFrame.")

In [ ]:
def update_download_state(trajectory_id, **fields):
    if not fields:
        return

    cur = conn.cursor()

    columns = ", ".join(f"{col} = ?" for col in fields.keys())
    params = list(fields.values()) + [trajectory_id]

    query = f"""
        UPDATE trajectories
        SET {columns},
            updated_at = CURRENT_TIMESTAMP
        WHERE trajectory_id = ?;
    """

    cur.execute(query, params)
    conn.commit()

    idx = trajectories.index[trajectories["trajectoryid"] == trajectory_id]

    if len(idx) == 0:
        print(f"Warning: trajectory_id {trajectory_id} not found in DataFrame.")
        return

    idx = idx[0]

    for col, val in fields.items():
        if col == "download_expires_at" and isinstance(val, str):
            val = pd.to_datetime(val)

        trajectories.at[idx, col] = val

In [ ]:
import requests
import shutil
import piexif
import tarfile
from PIL import Image
from tqdm import tqdm
import os


def get_download_id_and_size_in_bytes(trajectory_id):
    polygon = trajectories[
        trajectories["trajectoryid"] == trajectory_id
    ].geometry.values[0]
    coords = [[x, y] for x, y in polygon.exterior.coords]
    url = "https://mein.wien.gv.at/geodownload-backend/app/register"
    payload = {"data": {"coords": coords, "dataset": "KAPPAZUNDER 2020", "option": 2}}
    response = requests.post(url, json=payload)
    response.raise_for_status()
    items = response.json().get("items")
    return items.get("confirmation"), items.get("size") * 1024 * 1024


def request_confirm_email(download_id, email_address):
    with requests.Session() as session:
        session.get(
            f"https://mein.wien.gv.at/geodownload-ui/confirm/{download_id}",
            headers={
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36",
                "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
            },
        )
        response = session.patch(
            f"https://mein.wien.gv.at/geodownload-backend/app/confirm/{download_id}",
            json={"mail": email_address},
            headers={
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36",
                "Accept": "application/json, text/plain, */*",
                "Origin": "https://mein.wien.gv.at",
                "Referer": "https://mein.wien.gv.at/",
            },
        )
        response.raise_for_status()


def confirm_email():
    url = "https://mein.wien.gv.at/geodownload-backend/app/mail/2d4bf8b8-88cb-4c9c-b29b-bf2f5ef50c8f"
    response = requests.patch(url)
    response.raise_for_status()


def download(download_id, size):
    url = f"https://www.wien.gv.at/ogdgeodata/download/{download_id}.tar"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    with requests.get(url, headers=headers, stream=True) as r:
        r.raise_for_status()
        block_size = 8 * 1024
        progress_bar = tqdm(
            total=size,
            unit="iB",
            unit_scale=True,
            desc=f"Downloading {download_id}.tar",
        )
        with open(download_id + ".tar", "wb") as f:
            for chunk in r.iter_content(chunk_size=block_size):
                progress_bar.update(len(chunk))
                f.write(chunk)
        progress_bar.close()


def extract_and_remove_tar(download_id):
    tar_path = download_id + ".tar"
    with tarfile.open(tar_path, "r") as tar:
        tar.extractall(path=download_id)
    os.remove(tar_path)


def get_trajectory_dir_paths(download_id, trajectory_id):
    los_dirs = [f for f in os.listdir(download_id)]
    possible_trajectory_dir_paths = [
        os.path.join(
            download_id, los_dir, "Bild-Rohdaten", f"Trajektorie_{trajectory_id}"
        )
        for los_dir in los_dirs
    ]
    return [d for d in possible_trajectory_dir_paths if os.path.isdir(d)]


def prune_downloaded_data(download_id, trajectory_dir_paths_to_keep):
    for los_dir in os.listdir(download_id):
        bild_rohdaten_dir_path = os.path.join(download_id, los_dir, "Bild-Rohdaten")
        for trajectory_dir in os.listdir(bild_rohdaten_dir_path):
            trajectory_dir_path = os.path.join(bild_rohdaten_dir_path, trajectory_dir)
            if (
                os.path.isdir(trajectory_dir_path)
                and trajectory_dir_path not in trajectory_dir_paths_to_keep
            ):
                shutil.rmtree(trajectory_dir_path)


def remove_top_and_bottom_facing_images(trajectory_dir_paths):
    for trajectory_dir_path in trajectory_dir_paths:
        for name in os.listdir(trajectory_dir_path):
            if name.endswith(("0", "5")):
                shutil.rmtree(os.path.join(trajectory_dir_path, name))


def set_exif_tags(trajectory_dir_paths):
    def deg_to_dms_rational(deg_float):
        """
        Convert decimal degrees to EXIF DMS rational format.
        """
        deg_abs = abs(deg_float)
        deg = int(deg_abs)
        min_float = (deg_abs - deg) * 60
        min_ = int(min_float)
        sec = round((min_float - min_) * 60 * 10000)
        return ((deg, 1), (min_, 1), (sec, 10000))

    file_paths = []
    for trajectory_dir_path in trajectory_dir_paths:
        for sensor_dir in os.listdir(trajectory_dir_path):
            sensor_dir_path = os.path.join(trajectory_dir_path, sensor_dir)
            sensor_dir_content_paths = [
                os.path.join(sensor_dir_path, file)
                for file in os.listdir(sensor_dir_path)
            ]
            file_paths.extend(
                [file for file in sensor_dir_content_paths if os.path.isfile(file)]
            )

    for img_path in tqdm(file_paths, desc="Tagging images"):
        file = os.path.basename(img_path)
        row = points.loc[file]
        if row.empty:
            print(f"⚠️ No metadata found for {file}, skipping.")
            continue

        img = Image.open(img_path)

        # Load existing EXIF or create new
        exif_dict = piexif.load(img.info.get("exif", b""))

        # GPS
        exif_dict["GPS"] = {
            piexif.GPSIFD.GPSLatitudeRef: b"N",
            piexif.GPSIFD.GPSLatitude: deg_to_dms_rational(row["lat"]),
            piexif.GPSIFD.GPSLongitudeRef: b"E",
            piexif.GPSIFD.GPSLongitude: deg_to_dms_rational(row["lon"]),
        }

        # DateTimeOriginal
        exif_dict["Exif"][piexif.ExifIFD.DateTimeOriginal] = (
            row["epoch"].strftime("%Y:%m:%d %H:%M:%S").encode("utf-8")
        )

        # Insert EXIF back into image
        exif_bytes = piexif.dump(exif_dict)
        piexif.insert(exif_bytes, img_path)

In [ ]:
import imaplib
import email
from email.header import decode_header
import re


class ReadyToDownloadChecker:
    # Exact URL pattern with GUID
    GUID_REGEX = re.compile(
        r"https://www\.wien\.gv\.at/ogdgeodata/download/"
        r"([0-9a-f]{8}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{12})\.tar"
    )

    def __init__(self, email_address, app_password, email_subject_filter):
        self.email_address = email_address
        self.app_password = app_password
        self.subject_filter = email_subject_filter
        self.imap = None
        self.emails = []  # List of (subject, body)
        self._connect()
        self._fetch_emails_by_subject()

    def _connect(self):
        if self.imap:
            try:
                self.imap.logout()
            except:
                pass
        self.imap = imaplib.IMAP4_SSL("imap.gmail.com")
        self.imap.login(self.email_address, self.app_password)
        self.imap.select("inbox")

    def _fetch_emails_by_subject(self):
        search_criterion = f'(SUBJECT "{self.subject_filter}")'
        status, messages = self.imap.search(None, search_criterion)
        if status != "OK":
            return
        for num in messages[0].split():
            self._fetch_email(num)

    def _fetch_email(self, num):
        status, msg_data = self.imap.fetch(num, "(RFC822)")
        if status != "OK":
            return
        msg = email.message_from_bytes(msg_data[0][1])
        subject = decode_header(msg.get("Subject") or "")[0][0]
        if isinstance(subject, bytes):
            subject = subject.decode(errors="ignore")
        body = ""
        if msg.is_multipart():
            for part in msg.walk():
                if part.get_content_type() == "text/plain":
                    try:
                        body += part.get_payload(decode=True).decode(errors="ignore")
                    except:
                        pass
        else:
            try:
                body = msg.get_payload(decode=True).decode(errors="ignore")
            except:
                pass
        self.emails.append((subject, body))

    def refresh(self):
        """Fetch only unseen emails with matching subject"""
        try:
            self._connect()
            search_criterion = f'(UNSEEN SUBJECT "{self.subject_filter}")'
            status, messages = self.imap.search(None, search_criterion)
            if status == "OK":
                for num in messages[0].split():
                    self._fetch_email(num)
        except Exception as e:
            print("Error refreshing mail:", e)

    def get_ids(self):
        """Return a deduplicated list of GUIDs from cached emails"""
        guids = set()
        for _, body in self.emails:
            matches = self.GUID_REGEX.findall(body)
            guids.update(matches)
        return list(guids)

In [ ]:
from datetime import timedelta
from time import sleep
from urllib.error import HTTPError


MAPILLARY_USER = "eliasgander"
MAPILLARY_EMAIL = "daring.64tum@icloud.com"
MAPILLARY_PASSWORD = "cesjuD-6tyjmo-maqgif"
EMAIL_ADDRESS = "tomturbo657@gmail.com"
GMAIL_APP_PASSWORD = "kruf aahg aiuc iwtr"
DOWNLOAD_READY_SUBJECT = "Download-Link zu Ihren Geodaten"
VOLUME_SIZE_IN_BYTES = 40 * 1024 * 1024 * 1024  # 40GB
IS_DEBUG = False

if IS_DEBUG:
  update_download_state('17720', download_id='5276d431-a054-4a84-a38c-6dfbccefdef0', download_bytes=1403*1024*1024, download_expires_at=(pd.Timestamp.now() + timedelta(days=7)).isoformat())
  update_download_state('16101', download_id='56a27033-35ed-4c3c-ba14-704cc256efac', download_bytes=668*1024*1024, download_expires_at=(pd.Timestamp.now() + timedelta(days=7)).isoformat())
  update_download_state('16471', download_id='e4a3d58b-2c58-4991-8e98-ae7b635d25cf', download_bytes=720*1024*1024, download_expires_at=(pd.Timestamp.now() + timedelta(days=7)).isoformat())

!mapillary_tools authenticate --user_name {MAPILLARY_USER} --user_email {MAPILLARY_EMAIL} --user_password {MAPILLARY_PASSWORD}

ready_to_download = ReadyToDownloadChecker(EMAIL_ADDRESS, GMAIL_APP_PASSWORD, DOWNLOAD_READY_SUBJECT)

sensors_completed_column_names = [f"is_sensor{i}_completed" for i in range(1, 5)]

while not trajectories[sensors_completed_column_names].to_numpy().all():
    print(f"Number of uncompleted trajectories: {trajectories[~trajectories[sensors_completed_column_names].to_numpy().all(axis=1)].shape[0]}")

    expiring_trajectories = trajectories[trajectories['download_expires_at'].notna() & (trajectories['download_expires_at'] < pd.Timestamp.now() + timedelta(hours=5))]
    if not expiring_trajectories.empty:
        print(f"Resetting expiring trajectories with IDs: {expiring_trajectories['trajectoryid'].tolist()}")
        for trajectory_id in expiring_trajectories["trajectoryid"]:
          update_download_state(
              trajectory_id,
              download_id=None,
              download_bytes=None,
              download_expires_at=None
          )

    ready_to_download.refresh()
    trajectories_to_download = trajectories[trajectories['download_id'].isin(ready_to_download.get_ids())]
    if len(trajectories_to_download) <= 5:
        print(f"Only {len(trajectories_to_download)} downloadable trajectories left")
        trajectories_to_prepare = trajectories[trajectories['download_id'].isna()]
        if trajectories_to_prepare.empty:
            print("No trajectories left to prepare")
        else:
            trajectories_to_prepare = trajectories_to_prepare.sample(min(10, len(trajectories)))
            print(f"Preparing {len(trajectories_to_prepare)} trajectories with ids: {trajectories_to_prepare['trajectoryid'].tolist()}")
            for index, trajectory in trajectories_to_prepare.iterrows():
                trajectory_id = trajectory['trajectoryid']
                try:
                    download_id, size = get_download_id_and_size_in_bytes(trajectory_id)
                    if size > VOLUME_SIZE_IN_BYTES / 2 * 0.95:
                      print(f"Skipping trajectoryid {trajectory_id} with downloadid {download_id} because size {size} exceeds limit.")
                      continue
                    request_confirm_email(download_id, EMAIL_ADDRESS)
                    sleep(60)
                    confirm_email()
                    update_download_state(trajectory_id, download_id=download_id, download_bytes=size, download_expires_at=(pd.Timestamp.now() + timedelta(days=7)).isoformat())
                    print(f"Successfully prepared trajectoryid {trajectory_id} with downloadid {download_id}")
                except Exception as e:
                    print(f"Error preparing trajectoryid {trajectory_id} with downloadid {download_id}: {e}")

    if trajectories_to_download.empty:
        print("No trajectories ready for download. Sleeping five minutes.")
        sleep(300)
        continue

    trajectory_to_download = trajectories_to_download.sort_values('download_expires_at').iloc[0]
    trajectory_id = trajectory_to_download['trajectoryid']
    download_id = trajectory_to_download['download_id']
    if not os.path.isdir(download_id):
      try:
            download(download_id, trajectory_to_download['download_bytes'])
      except HTTPError as e:
          if e.code == 404:
            print(f"Resetting downloadid {download_id} of trajectory {trajectory_id} because 404 error: {e}")
            update_download_state(trajectory_id, download_id=None, download_bytes=None, download_expires_at=None)
          else:
            print(f"Sleeping for five minutes because download of trajectory {trajectory_id} with downloadid {download_id} failed with message: {e}")
            sleep(300)
          continue
      extract_and_remove_tar(download_id)
      
    trajectory_dir_paths = get_trajectory_dir_paths(download_id, trajectory_id.split('_', 1)[0])
    prune_downloaded_data(download_id, trajectory_dir_paths)
    remove_top_and_bottom_facing_images(trajectory_dir_paths)
    set_exif_tags(trajectory_dir_paths)
    for i in range(1, 5):
        if trajectories[trajectories['trajectoryid'] == trajectory_id][f'is_sensor{i}_completed'].iloc[0]:
          continue
        cmd = f"""
        mapillary_tools process_and_upload \
            --overwrite_all_EXIF_tags \
            --device_make Teledyne \
            --device_model Ladybug6 \
            --offset_angle {(i - 1) * 90} \
            --interpolate_directions \
            --user_name {MAPILLARY_USER} \
            --noresume \
            {download_id}/*/Bild-Rohdaten/Trajektorie_{trajectory_id}/Sensor_*{i}/
        """
        !{cmd}
        update_download_state(trajectory_id, **{f'is_sensor{i}_completed': 1})
    shutil.rmtree(download_id)


print("All trajectories completed.")